In [1]:
from new_loader import preprocess_and_save_audio_in_tensors
preprocess_and_save_audio_in_tensors()

10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed


In [1]:
from new_loader import preprocess_and_save_audio_in_tensors
preprocess_and_save_audio_in_tensors(denoise=True)

10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed
10 000 files proccessed


In [21]:
import os
import csv
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import efficientnet_b0
from tqdm import tqdm

from data_loading import TorchTensorFolderDataset
def get_loader(data_size:str="", target_data:str="train", batch_size:int=16):
    # path_to_raw = os.path.join(os.getcwd(), "data", "preprocessed", data_size, "raw", target_data)
    path_to_mel = os.path.join(os.getcwd(), "data", "preprocessed", data_size, "raw", target_data)
    
    # dataset = EnsembleDataset(path_to_raw, path_to_mel)
    dataset = TorchTensorFolderDataset(path_to_mel)
    # n = len(dataset.labels)
    n = len(dataset.class_to_idx)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=False)

    return loader, n

loader = get_loader()[0]

In [25]:
batch = next(iter(loader))
print(max(batch[0][0][0]))


tensor(0.1095)


In [6]:
from new_loader import generate_dataset_with_optional_augmented_sample
generate_dataset_with_optional_augmented_sample(denoise=False)

1200
1500
1751


In [2]:
import os
import random
import torch
from torch.utils.data import Dataset, DataLoader


class NoiseVoiceClassificationDataset(Dataset):
    def __init__(self, folder_noise, folder_voice, seed=42):
        self.samples = []

        # Wczytaj ścieżki noise i voice
        noise_paths = [os.path.join(folder_noise, f) for f in os.listdir(folder_noise) if f.endswith('.pt')]

        voice_paths = []
        for root, _, files in os.walk(folder_voice):
            for f in files:
                if f.endswith('.pt'):
                    voice_paths.append(os.path.join(root, f))

        # Ustal liczbę próbek
        num_noise = len(noise_paths)

        # Losuj voice z zachowaniem równowagi i deterministyczności
        rnd = random.Random(seed)
        voice_paths_sampled = rnd.sample(voice_paths, k=min(num_noise, len(voice_paths)))

        # Buduj listę próbek: (ścieżka, etykieta)
        self.samples.extend([(p, 0) for p in noise_paths])
        self.samples.extend([(p, 1) for p in voice_paths_sampled])

        # Opcjonalnie przemieszaj wszystkie próbki (zachowując seed)
        rnd.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        tensor = torch.load(path,weights_only=False)
        return tensor, torch.tensor(label, dtype=torch.long)





def load_dataloaders(noise_dir, voice_dir, seed=42, batchsize=16):
    splits = ['train', 'validation', 'test']
    loaders = {}

    for i, split in enumerate(splits):
        noise_folder = os.path.join(noise_dir, split)
        voice_folder = os.path.join(voice_dir, split)

        dataset = NoiseVoiceClassificationDataset(noise_folder, voice_folder, seed=seed + i)

        loaders[split] = DataLoader(
            dataset,
            batch_size=batchsize,
            shuffle=(True if split == 'train' else False)
        )

    return loaders['train'], loaders['validation'], loaders['test']

In [3]:
noise_dir = r"C:\Users\Michał\Documents\Projects\Deep_learning_projects\project_2\data\preprocessed\noise\standard\raw"
voice_dir = r"C:\Users\Michał\Documents\Projects\Deep_learning_projects\project_2\data\preprocessed\raw"
train,val,test = load_dataloaders(noise_dir,voice_dir, batchsize = 1)

In [4]:
from CNN_transfomers_implementation import Mel_transformer, RawAudioTransformer, train_transformer
train,val,test = load_dataloaders(noise_dir,voice_dir, batchsize = 16)

In [5]:
spect_model = Mel_transformer(num_classes=30)
spect_model = train_transformer(spect_model, train, val, epochs=20)


Epoch 1: Train Loss: 0.1193 | Train Acc: 0.9805 | Val Acc: 0.9971


Epoch 2: Train Loss: 0.0333 | Train Acc: 0.9924 | Val Acc: 0.9986


Epoch 3: Train Loss: 0.0128 | Train Acc: 0.9971 | Val Acc: 0.9986


Epoch 4: Train Loss: 0.0222 | Train Acc: 0.9952 | Val Acc: 1.0000


Epoch 5: Train Loss: 0.0120 | Train Acc: 0.9952 | Val Acc: 0.9971


Epoch 6: Train Loss: 0.0066 | Train Acc: 0.9986 | Val Acc: 1.0000


Epoch 7: Train Loss: 0.0015 | Train Acc: 1.0000 | Val Acc: 1.0000
Early stopping triggered at epoch 7
Best Val Acc: 1.0000


In [7]:
from CNN_transfomers_implementation import evaluate_model
evaluate_model(spect_model,test)

(1.0, 0.0011)